In [110]:
import eee

import pandas as pd
import numpy as np

from eee.ensemble import Ensemble
from eee.evolve.fitness import FitnessContainer
from eee.evolve.fitness import ff_on
from eee.evolve.fitness import ff_off
from eee.evolve.wright_fisher import wright_fisher


# Basic ensemble
ens = Ensemble(R=1)
ens.add_species(name="s1",
                dG0=0,
                observable=True)
ens.add_species(name="s2",
                dG0=0.167,
                observable=False,
                mu_stoich={"X":2})

mu_dict = {"X":[0,3.333]}

ddg_df = pd.DataFrame({"site":[1,1],
                       "mut":["A1V","A1P"],
                       "s1":[-1.677,0.167],
                       "s2":[3.333,-5000]})

fc = FitnessContainer(ens,
                      mu_dict,
                      [ff_on,ff_off],
                      select_on="fx_obs",
                      fitness_kwargs={},
                      T=1)

gc = eee.evolve.genotype.GenotypeContainer(fc=fc,
                                           ddg_df=ddg_df)


In [59]:
ens.get_obs(mut_energy={},mu_dict=mu_dict,T=1)

,T,X,s1,s2,fx_obs,dG_obs
0,1.0,0.000,0.541653,0.458347,0.541653,-0.167
1,1.0,3.333,0.001503,0.998497,0.001503,6.499


In [104]:
eee.evolve.fitness._fitness_function(ens=ens,
                                     mut_energy={},
                                     mu_dict=mu_dict,
                                     fitness_fcns=[ff_on,ff_off],
                                     select_on="fx_obs",
                                     fitness_kwargs={},
                                     T=1)

array([0.54165324, 0.99849732])

In [105]:

df = ens.get_obs(mu_dict=mu_dict,
                 T=1)
v = np.array(df["fx_obs"])
print(ff_on(v[0]),ff_off(v[1]))
df

0.5416532393676118 0.9984973180670926


,T,X,s1,s2,fx_obs,dG_obs
0,1.0,0.000,0.541653,0.458347,0.541653,-0.167
1,1.0,3.333,0.001503,0.998497,0.001503,6.499


In [108]:
df = ens.get_obs(mu_dict=mu_dict,
                 mut_energy={"s1":-1.677,"s2":3.333},
                 T=1)
v = np.array(df["fx_obs"])
print(ff_on(v[0]),ff_off(v[1]))
df


0.9943867704232219 0.8159281306107955


,T,X,s1,s2,fx_obs,dG_obs
0,1.0,0.000,0.994387,0.005613,0.994387,-5.177
1,1.0,3.333,0.184072,0.815928,0.184072,1.489


In [109]:
df = ens.get_obs(mu_dict=mu_dict,
                 mut_energy={"s1":0.167,"s2":-16.667},
                 T=1)
v = np.array(df["fx_obs"])
print(ff_on(v[0]),ff_off(v[1]))
df


5.775822590594909e-08 0.999999999926446


,T,X,s1,s2,fx_obs,dG_obs
0,1.0,0.000,5.775823e-08,1.0,5.775823e-08,16.667
1,1.0,3.333,7.355409e-11,1.0,7.355409e-11,23.333


In [64]:
print(fc.fitness({}))
print(fc.fitness({"s1":-0.167,"s2":3.333}))
print(fc.fitness({"s1":0.167,"s2":-16.667}))

0.5408393068309133
0.928795407551965
5.7758225901700734e-08


In [97]:
gc, generations = wright_fisher(gc=gc,
                                mutation_rate=0.001,
                                population=10000,
                                num_generations=1000,)

  0%|          | 0/999 [00:00<?, ?it/s]

In [98]:

genotypes = {}
for gen in generations:
    for g in gen:
        mut_genotype = "".join(gc.genotypes[g].mutations)
        if mut_genotype not in genotypes:
            genotypes[mut_genotype] = 0
        
        genotypes[mut_genotype] += gen[g]
        
assert genotypes["A1V"] > genotypes["A1P"]
print(genotypes)
        

{'': 173565, 'A1V': 9821432, 'A1P': 5003}


In [92]:
9821432

0.9900628059481937

In [82]:
print(84213/100000)
print(15733/100000)
print(54/100000)

0.84213
0.15733
0.00054


In [88]:
31964/10000000

0.0031964